In [ ]:
# ==========================================
# UNIVERSAL SETUP & DATA LOADER
# ==========================================
import os
import shutil
import gdown
from google.colab import drive

# 1. Mount Drive (For saving results later)
print("🔌 Mounting Google Drive...")
drive.mount('/content/drive')

# 2. Configuration
FILE_ID = '1aESAhJMB3ur3kpgqoUetGvQf6l2wuAiW'
DOWNLOAD_OUTPUT = '/content/Final_Dataset.zip'
EXTRACT_DIR = '/content/Final_Dataset_Extracted'

# 3. Download
if not os.path.exists(DOWNLOAD_OUTPUT):
    print(f"⬇️  Downloading dataset...")
    gdown.download(f'https://drive.google.com/uc?id={FILE_ID}', DOWNLOAD_OUTPUT, quiet=False)
else:
    print("✅ Zip file already exists.")

# 4. Extract
if not os.path.exists(EXTRACT_DIR):
    print(f"📦 Extracting...")
    shutil.unpack_archive(DOWNLOAD_OUTPUT, EXTRACT_DIR)
else:
    print("✅ Already extracted.")

# 5. Locate the actual Data Root (Fixing the nesting issue)
DATASET_ROOT = os.path.join(EXTRACT_DIR, 'Final_Dataset', 'Final_Dataset')
if not os.path.exists(os.path.join(DATASET_ROOT, 'Train')):
    # Fallback search if folder structure changes
    for root, dirs, files in os.walk(EXTRACT_DIR):
        if 'Train' in dirs:
            DATASET_ROOT = root
            break

print(f"✅ DATASET READY AT: {DATASET_ROOT}")
# Save this path to a global variable for the training scripts
os.environ['NXP_DATASET_ROOT'] = DATASET_ROOT

🔌 Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
⬇️  Downloading dataset...


Downloading...
From (original): https://drive.google.com/uc?id=1aESAhJMB3ur3kpgqoUetGvQf6l2wuAiW
From (redirected): https://drive.google.com/uc?id=1aESAhJMB3ur3kpgqoUetGvQf6l2wuAiW&confirm=t&uuid=31ebb0ca-286e-4398-8d11-7bfe7034c7b3
To: /content/Final_Dataset.zip
100%|██████████| 810M/810M [00:12<00:00, 66.4MB/s]


📦 Extracting...
✅ DATASET READY AT: /content/Final_Dataset_Extracted/Final_Dataset


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models, Input, Model
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# --- CONFIG ---
MODEL_NAME = "MobileNetV2_Focal_HardMiner"
DATA_DIR = os.environ.get('NXP_DATASET_ROOT')

BASE_DIR = f'./NXP_Results_{MODEL_NAME}'
CKPT_DIR = os.path.join(BASE_DIR, 'Checkpoints')
MODEL_DIR = os.path.join(BASE_DIR, 'Model_Files')
REPORT_DIR = os.path.join(BASE_DIR, 'Reports')
for d in [CKPT_DIR, MODEL_DIR, REPORT_DIR]: os.makedirs(d, exist_ok=True)

# 224x224 is MANDATORY for seeing 1-pixel bridges
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 25

# --- DATA ---
train_ds = tf.keras.utils.image_dataset_from_directory(os.path.join(DATA_DIR, 'Train'), image_size=IMG_SIZE, batch_size=BATCH_SIZE, color_mode='grayscale', shuffle=True, seed=123)
val_ds = tf.keras.utils.image_dataset_from_directory(os.path.join(DATA_DIR, 'Validation'), image_size=IMG_SIZE, batch_size=BATCH_SIZE, color_mode='grayscale', shuffle=False)
class_names = train_ds.class_names

# --- 1. THE SECRET WEAPON: FOCAL LOSS ---
def focal_loss(gamma=4.0, alpha=0.25):
    # gamma: Higher = Focus more on hard examples (Bridges/Opens)
    # alpha: Balance factor for classes
    def focal_loss_fixed(y_true, y_pred):
        # Sparse to One-Hot
        y_true = tf.one_hot(tf.cast(y_true, tf.int32), depth=len(class_names))
        y_true = tf.cast(y_true, tf.float32)

        # Clip to prevent NaN
        y_pred = tf.clip_by_value(y_pred, 1e-7, 1. - 1e-7)

        # Calculate Cross Entropy
        cross_entropy = -y_true * tf.math.log(y_pred)

        # Calculate Focal Weight: (1 - p)^gamma
        weight = alpha * y_true * tf.pow((1 - y_pred), gamma)

        # Final Loss
        loss = weight * cross_entropy
        return tf.reduce_sum(loss, axis=1)
    return focal_loss_fixed

# --- MODEL ---
def build_model():
    inputs = Input(shape=IMG_SIZE + (1,))
    x = layers.Rescaling(1./255)(inputs)

    # Simple Augmentation (Since dataset is already rotated)
    x = layers.RandomFlip("horizontal_and_vertical")(x)
    x = layers.RandomContrast(0.2)(x) # Helps distinct Bridges from Background

    # Virtual RGB
    x = layers.Conv2D(3, 1, padding='same', use_bias=False)(x)

    # Load MobileNetV2 (0.75)
    base_model = tf.keras.applications.MobileNetV2(
        input_shape=(224, 224, 3),
        include_top=False,
        weights='imagenet',
        alpha=0.75
    )

    # PARTIAL UNFREEZE:
    # We freeze the bottom (shapes) and unfreeze the top (textures/details)
    base_model.trainable = True
    # Freeze the first 100 layers (Structural features like lines/circles)
    # Unfreeze the last 50 layers (Specific features like "Jagged Lines")
    for layer in base_model.layers[:-50]:
        layer.trainable = False

    x = base_model(x)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.2)(x)
    # Softmax is REQUIRED for Focal Loss stability
    outputs = layers.Dense(len(class_names), activation='softmax')(x)

    return Model(inputs, outputs)

model = build_model()

# Low LR because we are fine-tuning
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

model.compile(
    optimizer=optimizer,
    loss=focal_loss(gamma=4.0, alpha=0.25), # Using the Custom Loss
    metrics=['accuracy']
)

# --- TRAIN ---
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(CKPT_DIR, "best_model.keras"), save_best_only=True, monitor='val_accuracy'),
    tf.keras.callbacks.CSVLogger(os.path.join(REPORT_DIR, "log.csv"))
]

print(f"🚀 Training with Focal Loss (Gamma=4.0) to hunt Bridges/Opens...")
history = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS, callbacks=callbacks)

# --- REPORT ---
model.load_weights(os.path.join(CKPT_DIR, "best_model.keras"))
y_pred = np.argmax(model.predict(val_ds), axis=1)
y_true = np.concatenate([y for x, y in val_ds], axis=0)

print("\n--- CLASSIFICATION REPORT ---")
print(classification_report(y_true, y_pred, target_names=class_names))
with open(os.path.join(REPORT_DIR, "report.txt"), "w") as f: f.write(classification_report(y_true, y_pred, target_names=class_names))

print("💾 Converting to TFLite...")
def rep_data():
    for img, _ in train_ds.take(100): yield [tf.cast(img, tf.float32)]

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = rep_data
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
tflite_model = converter.convert()

with open(os.path.join(MODEL_DIR, f"{MODEL_NAME}.tflite"), 'wb') as f: f.write(tflite_model)

hex_lines = [f"0x{b:02x}" for b in tflite_model]
cc_code = f'#include "{MODEL_NAME}.h"\nconst unsigned int g_model_len = {len(tflite_model)};\nconst unsigned char g_model[] __attribute__((aligned(16))) = {{\n' + ", ".join(hex_lines) + "\n};\n"
with open(os.path.join(MODEL_DIR, f"{MODEL_NAME}.cc"), 'w') as f: f.write(cc_code)
with open(os.path.join(MODEL_DIR, f"{MODEL_NAME}.h"), 'w') as f: f.write(f'extern const unsigned int g_model_len;\nextern const unsigned char g_model[];\n')

print(f"✅ {MODEL_NAME} Complete!")

Found 15996 files belonging to 8 classes.
Found 4433 files belonging to 8 classes.
5903360/5903360 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
🚀 Training with Focal Loss (Gamma=4.0) to hunt Bridges/Opens...
Epoch 1/25
500/500 ━━━━━━━━━━━━━━━━━━━━ 113s 145ms/step - accuracy: 0.6812 - loss: 0.0959 - val_accuracy: 0.3521 - val_loss: 0.3829
Epoch 2/25
500/500 ━━━━━━━━━━━━━━━━━━━━ 32s 64ms/step - accuracy: 0.8680 - loss: 0.0218 - val_accuracy: 0.4893 - val_loss: 0.4401
Epoch 3/25
500/500 ━━━━━━━━━━━━━━━━━━━━ 32s 64ms/step - accuracy: 0.9048 - loss: 0.0143 - val_accuracy: 0.7072 - val_loss: 0.1027
Epoch 4/25
500/500 ━━━━━━━━━━━━━━━━━━━━ 33s 66ms/step - accuracy: 0.9183 - loss: 0.0108 - val_accuracy: 0.7882 - val_loss: 0.0502
Epoch 5/25
500/500 ━━━━━━━━━━━━━━━━━━━━ 32s 63ms/step - accuracy: 0.9322 - loss: 0.0084 - val_accuracy: 0.8238 - val_loss: 0.0430
Epoch 6/25
500/500 ━━━━━━━━━━━━━━━━━━━━ 34s 68ms/step - accuracy: 0.9388 - loss: 0.0069 - val_accuracy: 0.8642 - val_loss: 0.0231
Epoch 7/25
500/500 ━━━

/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/convert.py:854: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


✅ MobileNetV2_Focal_HardMiner Complete!
